<a href="https://colab.research.google.com/github/kundanmishra/MachineLearning/blob/master/NLP_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing required libraries

In [3]:
# Importing required libraries
# conda install -c conda-forge scikit-plot
# conda install -c conda-forge wordcloud
import json
import tensorflow as tf
import requests
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt;

# Get the dataset

In [4]:
# Get the dataset
pd.set_option('max_colwidth', 100)
sarcasm = pd.read_json("./Sarcasm_Headlines_Dataset_v2.json", lines=True)
sarcasm.head()


ValueError: ignored

In [ ]:
sarcasm.info()

In [ ]:
sarcasm.describe()

# Some basic EDA

In [ ]:
sns.set_style("dark")
sns.countplot(sarcasm.is_sarcastic)

In [ ]:
from wordcloud import WordCloud,STOPWORDS

In [ ]:
# Text that is Not Sarcastic
plt.figure(figsize = (20,20)) 
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800).generate(" ".join(sarcasm[sarcasm.is_sarcastic == 0].headline))
plt.imshow(wc , interpolation = 'bilinear')

In [ ]:
# Text that is Sarcastic
plt.figure(figsize = (20,20)) 
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800).generate(" ".join(sarcasm[sarcasm.is_sarcastic == 1].headline))
plt.imshow(wc , interpolation = 'bilinear')

# Removing punctuation

In [ ]:
import string

def no_punctuation(text):
    no_punc = "".join([i for i in text if i not in string.punctuation])
    return no_punc

In [ ]:
sarcasm['headline'] = sarcasm['headline'].apply(lambda x : no_punctuation(x))
sarcasm['headline'].head(10)

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
sarcasm['headline'] = sarcasm['headline'].apply(lambda x : tokenizer.tokenize(x.lower()))
sarcasm['headline'].head(10)

# remove_stopwords

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
def remove_stopwords(text):
    text=[word for word in text if word not in stopwords.words('english')]
    return text

In [ ]:
sarcasm['headline'] = sarcasm['headline'].apply(lambda x : remove_stopwords(x))
sarcasm['headline'].head(10)

# WordNetLemmatizer

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
def lemmalize(text):
    lem_lext = " ".join([lemmatizer.lemmatize(word) for word in text])
    return lem_lext

In [ ]:
sarcasm['headline'] = sarcasm['headline'].apply(lambda x : lemmalize(x))
sarcasm['headline'].head(10)

In [ ]:
sarcastic = sarcasm['headline'].loc[sarcasm['is_sarcastic'] == 1]
acclaim = sarcasm['headline'].loc[sarcasm['is_sarcastic'] == 0]

In [ ]:
sarcastic.reset_index(inplace = True, drop = True)
acclaim.reset_index(inplace = True, drop = True)

In [ ]:
sarcastic.head(),acclaim.head()

# Create a Bag of Words

In [ ]:
# Create two objects X and y. X will be the 'headline' column of sarcasm dataframe and y will 
# be the 'is_sarcastic' column of yelp. create a CountVectorizer object and split the data into training 
# and testing sets. Train a MultinomialNB model for classifying the headlines and Display the confusion Matrix 

In [ ]:
count_vec = CountVectorizer()
bow = count_vec.fit_transform(sarcasm['headline'])
bow = np.array(bow.todense())

In [ ]:
X = bow
y = sarcasm['is_sarcastic']

# Split the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    stratify=y)

# Fit the model

In [ ]:
# Fit the model
model = MultinomialNB().fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate model performance

In [ ]:
# Evaluate model performance
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred, average="macro"))
print('ROC AUC:', roc_auc_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

# confusion matrix

In [ ]:
# Constructing the confusion matrix.
import scikitplot as skplt
skplt.metrics.plot_confusion_matrix(y_test, y_pred)

# Where did it go wrong? - Observing the prediction of the model. 

In [ ]:
#Where did it go wrong?
results = pd.DataFrame(data={'predicted': y_pred, 'actual': y_test})
predictions = results.join(sarcasm)

In [ ]:
def is_correct(predicted, actual):
    if predicted == actual:
        return True
    else:
        return False

predictions['correct'] = predictions.apply(lambda x: is_correct(x.predicted, x.actual), axis=1)
predictions = predictions[['headline','predicted','actual','correct']]

In [ ]:
predictions[predictions['correct']==False].sample(10)

# HMM POS tagging on the first 4 rows of ‘headline’

In [ ]:
#!python -m spacy download en_core_web_md
#!python -m spacy download en_core_web_lg
import spacy
nlp = spacy.load("en_core_web_sm")



In [ ]:
import spacy
nlp = spacy.load('en')

In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_lg")

In [ ]:
for i in range(4):
     print(sarcasm['headline'][i])